In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import os
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
MODEL_NAME = os.getenv("MODEL_NAME")
MODEL_PATH = os.getenv("MODEL_PATH")
MODEL_OUTPUT_DIR = os.getenv("MODEL_OUTPUT_DIR")
MODEL_ADOPTOR_REPO_ID = os.getenv("MODEL_ADOPTOR_REPO_ID")
MERGED_MODEL_OUTPUT_DIR = os.getenv("MERGED_MODEL_OUTPUT_DIR")
MODEL_FINETUNED_REPO_ID = os.getenv("MODEL_FINETUNED_REPO_ID")

In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

In [16]:
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    # attn_implementation="flash_attention_2",
    cache_dir=MODEL_PATH,
)

In [17]:
# Load LoRA adapters

# Give the path to the LoRA adapter or the Hugging Face repo ID

model = PeftModel.from_pretrained(base_model, MODEL_ADOPTOR_REPO_ID) # type: ignore

In [18]:
# Merge LoRA into base model (this makes the adapters permanent)
model = model.merge_and_unload() # type: ignore


In [8]:
# Save merged model and tokenizer
model.save_pretrained(MERGED_MODEL_OUTPUT_DIR)
tokenizer.save_pretrained(MERGED_MODEL_OUTPUT_DIR)

print(f"✅ Merged model saved at: {MERGED_MODEL_OUTPUT_DIR}")

✅ Merged model saved at: ../../Models/Qwen2.5-0.5B-Instruct-RBI-QA-Merged


In [19]:
model.push_to_hub(
    repo_id=MODEL_FINETUNED_REPO_ID,)
tokenizer.push_to_hub(
    repo_id=MODEL_FINETUNED_REPO_ID,)

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vishva007/Qwen2.5-0.5B-Instruct-RBI-QA-Finetuned/commit/19a97a9c66a2cbafba49c5cfeb0135ec4ec86109', commit_message='Upload tokenizer', commit_description='', oid='19a97a9c66a2cbafba49c5cfeb0135ec4ec86109', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Vishva007/Qwen2.5-0.5B-Instruct-RBI-QA-Finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Vishva007/Qwen2.5-0.5B-Instruct-RBI-QA-Finetuned'), pr_revision=None, pr_num=None)